# Transcript extraction

## Audio Transcript

### Using Whisper

In [1]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=386f29423506b8157f32d32ed67856e360efe57e976b4a4cc7df0e9ee2a49ba3
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


### Audio extraction

In [2]:
from moviepy.editor import VideoFileClip

def extract_audio(input_file, output_file):
    try:
        video = VideoFileClip(input_file)
        audio = video.audio
        audio.write_audiofile(output_file, codec='libmp3lame')  # Specify the codec for MP3 format
        print("Audio extracted successfully!")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # input_file = "/content/1.mp4"  # Change this to your input video file
    input_file = "/content/1.mp4"  # Change this to your input video file
    output_file = "/content/1.mp3"  # Change this to the desired output audio file

    extract_audio(input_file, output_file)


  if event.key is 'enter':



MoviePy - Writing audio in /content/1.mp3


MoviePy - Done.
Audio extracted successfully!


In [3]:
import whisper
import warnings
import math
warnings.simplefilter("ignore")

def create_timestamps(segments):
    timestamps = []
    for segment in segments:
        start_time = segment['start']
        end_time = segment['end']
        timestamps.append((start_time, end_time))
    return timestamps

def format_timestamp(seconds):
    minutes = math.floor(seconds / 60)
    seconds = math.floor(seconds % 60)
    return f"{minutes:02}:{seconds:02}"

model = whisper.load_model("tiny")
# Transcribe audio
result = model.transcribe(audio="/content/1.mp3", task='transcription')

# Check the keys in the result dictionary
print("Keys in result:", result.keys())

# Extract segments from the result
segments = result['segments']
print("Segments:", segments)

# Create timestamps from segments
timestamps = create_timestamps(segments)
print("Timestamps:", timestamps)

# Initialize a variable to hold the transcript
audio_transcript = ""

# Print transcribed text with formatted timestamps and save it into audio_transcript
for i, (segment, (start_time, end_time)) in enumerate(zip(segments, timestamps)):
    text = segment['text']
    start_formatted = format_timestamp(start_time)
    end_formatted = format_timestamp(end_time)
    audio_transcript += f"Timestamps ({start_formatted} - {end_formatted}): {text}\n"

# Print the complete audio transcript
print("Audio Transcript:\n", audio_transcript)


100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 38.5MiB/s]


Keys in result: dict_keys(['text', 'segments', 'language'])
Segments: [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.0, 'text': ' Hello everybody, this is the Racing Rashput welcome to Magnet Bains.', 'tokens': [50364, 2425, 2201, 11, 341, 307, 264, 38832, 46298, 2582, 2928, 281, 19664, 302, 363, 2315, 13, 50564], 'temperature': 0.0, 'avg_logprob': -0.6969393481005419, 'compression_ratio': 1.8284518828451883, 'no_speech_prob': 0.1413344293832779}, {'id': 1, 'seek': 0, 'start': 4.0, 'end': 7.0, 'text': ' This video is going to start from a new chapter.', 'tokens': [50564, 639, 960, 307, 516, 281, 722, 490, 257, 777, 7187, 13, 50714], 'temperature': 0.0, 'avg_logprob': -0.6969393481005419, 'compression_ratio': 1.8284518828451883, 'no_speech_prob': 0.1413344293832779}, {'id': 2, 'seek': 0, 'start': 7.0, 'end': 12.0, 'text': ' Class 8 history chapter, chapter 1, ruling the countryside.', 'tokens': [50714, 9471, 1649, 2503, 7187, 11, 7187, 502, 11, 21437, 264, 28252, 13, 50964], 'temperature'

## Transcript extracted from Image Capturing

### Extracting Text from Image

In [4]:
import cv2

def extract_frames(video_path, start_time, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    frame_count = 0

    # Total frames to skip for 30-second intervals
    frame_skip = frame_interval * fps

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Start processing frames after the specified start time (in seconds)
        if frame_count >= start_time * fps and frame_count % frame_skip == 0:
            frames.append(frame)

        frame_count += 1

    cap.release()
    return frames


In [5]:
!pip install ultralytics==8.3.40

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 12.0 MB/s eta 0:00:00


In [6]:
from ultralytics import YOLO

def detect_teacher_in_frame(model, frame):
    results = model(frame)
    for r in results:
        for box in r.boxes:
            label = box.cls[0]
            if label == 'person':  # Assuming 'person' class is indexed as the teacher
                return box.xyxy[0].tolist()  # Return bounding box of the teacher
    return None


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
def select_keyframes(frames, model):
    """
    Selects keyframes for each slide, picking frames where the teacher covers the least amount of text.

    Args:
    - frames: A list of frames extracted from the video.
    - model: YOLO model used for teacher detection.

    Returns:
    - keyframes: A list of selected keyframes, one for each detected slide.
    """
    keyframes = []
    current_slide_frames = []
    min_teacher_area = float('inf')
    best_frame = None

    for frame in frames:
        teacher_box = detect_teacher_in_frame(model, frame)

        if teacher_box:
            # Calculate teacher area in the frame
            x1, y1, x2, y2 = teacher_box
            area = (x2 - x1) * (y2 - y1)

            if area < min_teacher_area:
                min_teacher_area = area
                best_frame = frame

            # If the frame belongs to the same slide, accumulate it
            current_slide_frames.append(frame)
        else:
            # If no teacher detected, assume a slide change
            if best_frame:
                keyframes.append(best_frame)  # Save the best frame for the current slide
            else:
                keyframes.append(frame)  # Save the frame if no teacher detected

            # Reset for the next slide
            current_slide_frames = []
            min_teacher_area = float('inf')
            best_frame = None

    # Add the last slide's keyframe if not already added
    if best_frame:
        keyframes.append(best_frame)

    return keyframes


In [8]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (3,004 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [9]:
!pip install pytesseract

In [10]:
import re
import numpy as np
import pytesseract

def clean_ocr_text(text):
    """
    Cleans the OCR text by removing unwanted characters, extra spaces,
    random numbers, and correcting any known OCR errors.
    """
    # Remove any standalone numbers, special characters, and unwanted symbols
    text = re.sub(r'\b\d+\b', '', text)  # Remove isolated numbers
    text = re.sub(r'[^\w\s.,!?-]', '', text)  # Keep only words, spaces, and basic punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces or newlines with a single space
    text = text.strip()  # Remove leading/trailing spaces

    return text

def format_text_with_bullets(text):
    """
    Formats the cleaned text by splitting it into sentences and adding bullet points,
    with multiple line spaces between each sentence.
    """
    # Clean the text before formatting
    text = clean_ocr_text(text)

    # Split text into sentences using punctuation as delimiters
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Add bullet points and separate sentences with three line breaks
    formatted_text = "\n\n\n".join([f"- {sentence.strip()}" for sentence in sentences if sentence.strip()])

    return formatted_text

def extract_text_from_frame(frame):
    """
    Extracts text from the given frame using pytesseract OCR, then cleans
    and formats it by adding bullet points and separating sentences.
    """

    if frame is None or not isinstance(frame, np.ndarray):
        print("Error: Invalid frame.")
        return ""

    # Convert the frame to grayscale for better OCR performance
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply OCR to extract text from the frame
    text = pytesseract.image_to_string(gray)

    # Format the OCR text by adding bullet points and multiple line spaces
    formatted_text = format_text_with_bullets(text)

    return formatted_text


In [11]:
def process_video(video_path, start_time=30, frame_interval=5):
    # Load the YOLO model
    model = YOLO("yolov8n.pt")

    # Step 1: Extract frames after 30 seconds
    frames = extract_frames(video_path, start_time, frame_interval)

    # Step 2: Detect and select keyframes with the least obstruction using the YOLO model
    keyframes = select_keyframes(frames, model)

    # Step 3: Extract and combine text from multiple keyframes using OCR
    extracted_texts = []

    for keyframe in keyframes:
        if keyframe is not None:
            text = extract_text_from_frame(keyframe)
            if text:
                extracted_texts.append(text)

    # Combine all the extracted texts
    combined_text = "\n\n\n".join(extracted_texts)

    return combined_text

# Example Usage:
video_path = "/content/1.mp4"
# video_path = "/content/2.mp4"
extracted_text = process_video(video_path)
print(extracted_text)


100%|██████████| 6.25M/6.25M [00:00<00:00, 50.3MB/s]



0: 384x640 5 persons, 261.6ms
Speed: 7.7ms preprocess, 261.6ms inference, 24.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 468.5ms
Speed: 2.5ms preprocess, 468.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 340.0ms
Speed: 1.9ms preprocess, 340.0ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 remote, 204.5ms
Speed: 4.7ms preprocess, 204.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 205.7ms
Speed: 2.0ms preprocess, 205.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 212.7ms
Speed: 2.0ms preprocess, 212.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 231.1ms
Speed: 1.9ms preprocess, 231.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 227.6ms
Speed: 2.0ms preprocess, 227.6ms inference, 1.4ms postproce

In [12]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    # Clean text of extra spaces
    text = " ".join(text.split())  # Normalize spaces
    return text

def extract_key_points(text):
    # Apply NLP processing
    doc = nlp(text)

    key_points = []

    # Loop through sentences
    for sent in doc.sents:
        # Check for entities or key words (nouns/verbs)
        entities = [ent.text for ent in sent.ents]
        key_words = [token.lemma_ for token in sent if token.pos_ in ['NOUN', 'VERB', 'PROPN']]

        # If the sentence contains entities or key words, consider it important
        if entities or key_words:
            key_points.append(sent.text.strip())

    return key_points

def remove_duplicate_sentences(sentences, threshold=0.8):
    # Vectorize the sentences
    vectorizer = TfidfVectorizer().fit_transform(sentences)
    vectors = vectorizer.toarray()

    # Calculate similarity between each pair of sentences
    similarity_matrix = cosine_similarity(vectors)

    # List to store unique sentences
    unique_sentences = []

    # Track whether a sentence is too similar to an earlier one
    for i, sentence in enumerate(sentences):
        if not any(similarity_matrix[i][j] > threshold for j in range(i)):
            unique_sentences.append(sentence)

    return unique_sentences

def format_output(points):
    # Format the extracted points as bullet points
    formatted_points = [f"- {point}" for point in points]
    return "\n".join(formatted_points)

# Example raw text
raw_text = extracted_text

# Process the text
cleaned_text = clean_text(raw_text)
key_points = extract_key_points(cleaned_text)
unique_points = remove_duplicate_sentences(key_points, threshold=0.8)
formatted_output = format_output(unique_points)

print(formatted_output)


- - PUN Topas eeel iil Introduction Relate the story with previous chapter.
- - Battle of Plassey, Bauxar Connect with ruling the countryside - oN fORY - CHAPTER - RULING THE COUNTRYSIDE Introduction Relate the story with previous chapter.
- - Battle of Plassey, Bauxar Connect with ruling the cou pal ruler in - oN fORY - CHAPTER - RULING
- THE COUNTRYSIDE Introduction Relate the story with previous chapter.
- - Battle of Plassey, Bauxar Jo the Mughal ruler in - PUR cpio lls Introduction Relate the story with previous chapter.- Battle of Plassey, Bauxar Connect with ruling t sa from .
- - ruler in , Robert Clive accepting the Di - oN fORY - CHAPTER - RULING THE COUNTRYSIDE Introduction Relate the story with previous chapter.
- - Battle of Plassey, Bauxar Connect with intryside Robert Clive accept Bn the Mughal ulin - PUR cutee ils Introduction Relate the story with previous chapter.
- - Battle of Plassey, Bauxar Connect with ruling th Robert Clive accepting the Diwani of Ey ye Mughal ru

In [13]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


In [14]:
!pip install transformers sentence-transformers langchain langchain-community langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langcha

In [15]:
import os

from langchain_openai import ChatOpenAI # pass the model to use
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS # perform similarity search on vector embeddings
from langchain.embeddings import HuggingFaceEmbeddings # convert text into vector embeddings
from langchain.text_splitter import CharacterTextSplitter # split text or doc in to smaller chunks
from langchain.chains import RetrievalQA # retrieveing information from the embeddings

In [16]:
# os.environ["OPENAI_API_KEY"]=""

In [17]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (213 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123679 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [18]:
!pip install --upgrade nltk

In [19]:
# Assuming the input text is stored in a variable called `input_text`

# Input text (replace this with your desired string)
input_text = formatted_output

# Split the text into chunks
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=20)
documents = [Document(page_content=input_text)]  # Using Document object to simulate document structure
texts = text_splitter.split_documents(documents)

# Loading vector embedding model
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()
kb = FAISS.from_documents(texts, embeddings)

# Initialize OpenAI LLM for context-based processing
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Function to use LLM to correct spellings, remove duplicates, and format output
def process_text_with_llm(input_text):
    prompt = (
        "You are given the following text extracted from an image. "
        "Please correct any spelling mistakes, remove duplicate points, and provide a neat and concise formatted version of the text:\n\n"
        f"{input_text}\n\n"
        "Return the corrected and formatted text."
    )
    response = llm([HumanMessage(content=prompt)])
    return response.content

# Process the input text
formatted_result = process_text_with_llm(input_text)
print(formatted_result)


  embeddings = HuggingFaceEmbeddings()

  embeddings = HuggingFaceEmbeddings()



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

  response = llm([HumanMessage(content=prompt)])



- Introduction: Relate the story with previous chapter.
- Battle of Plassey, Bauxar: Connect with ruling the countryside
- Chapter: Ruling the Countryside
- Battle of Plassey, Bauxar: Connect with ruling the local ruler
- Battle of Plassey, Bauxar: Joins the Mughal ruler
- Robert Clive accepting the Diwani of the Mughal ruler
- Connect with ruling the countryside: Robert Clive accepting the Diwani of Bengal
- Connect with ruling the country: Robert Clive accepting the Diwani of Bengal, Bihar and Orissa from the Mughal ruler
- Connect with ruling the countryside: Robert Clive accepting the Diwani of Bengal, Bihar and Orissa
- What we are going to study in this chapter:
- The company becomes a Diwan
- Revenue for the company from agriculture
- Problems and solutions: Mahalwari and Ryotwari system
- Crops for Europe: Indigo
- The Blue Rebellion and after
- The company becomes a Diwan: Revenue for the company
- Problems and solutions: Mahalwari and Ryotwari system
- Crops for Europe: Indig

In [20]:
def remove_duplicates(input_text):
    lines = input_text.split('\n')
    unique_lines = []
    seen = set()

    for line in lines:
        line_cleaned = line.strip()
        if line_cleaned and line_cleaned not in seen:
            unique_lines.append(line)
            seen.add(line_cleaned)

    return '\n'.join(unique_lines)

# Removing duplicates
formatted_result = remove_duplicates(formatted_result)

# Output the unique text
print("Unique Text:\n", formatted_result)

Unique Text:
 - Introduction: Relate the story with previous chapter.
- Battle of Plassey, Bauxar: Connect with ruling the countryside
- Chapter: Ruling the Countryside
- Battle of Plassey, Bauxar: Connect with ruling the local ruler
- Battle of Plassey, Bauxar: Joins the Mughal ruler
- Robert Clive accepting the Diwani of the Mughal ruler
- Connect with ruling the countryside: Robert Clive accepting the Diwani of Bengal
- Connect with ruling the country: Robert Clive accepting the Diwani of Bengal, Bihar and Orissa from the Mughal ruler
- Connect with ruling the countryside: Robert Clive accepting the Diwani of Bengal, Bihar and Orissa
- What we are going to study in this chapter:
- The company becomes a Diwan
- Revenue for the company from agriculture
- Problems and solutions: Mahalwari and Ryotwari system
- Crops for Europe: Indigo
- The Blue Rebellion and after
- The company becomes a Diwan: Revenue for the company


## Combining Audio Transcript and Image Transcript

In [24]:
# Function to clean and combine transcripts from audio and images
def combine_transcripts(audio_transcript, image_transcript):
    # Cleaning up transcripts by removing extra spaces and line breaks
    audio_transcript_clean = re.sub(r'\s+', ' ', audio_transcript).strip()
    image_transcript_clean = re.sub(r'\s+', ' ', image_transcript).strip()

    # Combining transcripts
    combined_transcript = f"Audio Transcript: {audio_transcript_clean}\n\nImage Transcript: {image_transcript_clean}"
    return combined_transcript

# Example usage with sample transcripts
# audio_transcript = "This is an example audio transcript extracted using Whisper."
image_transcript = formatted_result
combined_transcript = combine_transcripts(audio_transcript, image_transcript)

In [25]:
print(combined_transcript)

Audio Transcript: Timestamps (00:00 - 00:04): Hello everybody, this is the Racing Rashput welcome to Magnet Bains. Timestamps (00:04 - 00:07): This video is going to start from a new chapter. Timestamps (00:07 - 00:12): Class 8 history chapter, chapter 1, ruling the countryside. Timestamps (00:12 - 00:14): The interesting chapter is going to be, Timestamps (00:14 - 00:18): now we have to connect the class 8 history in the class 8 history. Timestamps (00:18 - 00:22): We have seen a basic outline of history in our history of modern history. Timestamps (00:22 - 00:26): When the British are here, but the modern one is not seen in the modern world, Timestamps (00:26 - 00:30): so the western world context in the history of the colonial history there is no other idea. Timestamps (00:30 - 00:33): First of all, we have not seen the British's leading dynasty. Timestamps (00:33 - 00:37): We had come to trade, but the trade led to the territory. Timestamps (00:37 - 00:40): So Jake went to trade an

# Summarisation/RAG Implementation

In [26]:
!pip install transformers sentence-transformers langchain langchain-community langchain-openai faiss-cpu unstructured unstructured[pdf]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.9/486.9 kB 27.6 MB/s et

In [27]:
import os
import re
import json
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI

In [28]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [29]:
# Splitting the combined transcript into smaller chunks for RAG
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_text(combined_transcript)

In [30]:
# Creating a FAISS vector store for retrieval
embeddings = HuggingFaceEmbeddings()
vector_store = FAISS.from_texts(documents, embeddings)

  embeddings = HuggingFaceEmbeddings()



In [31]:
# Setting up the RAG model
retriever = vector_store.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [32]:
# Generating a summary of the video
query = '''Provide a summary with appropriate headers, and explain the concepts under each header in detail including all keywords.
The generated summary should follow the format as shown below:

**Introduction (starting timestamp):**
- Provide a brief overview of the chapter or section, mentioning the key topic or theme being discussed.

**Subsection Title (starting timestamp):**
- Clearly mention the key points related to this subsection.
- Ensure to include specific events or important dates if relevant.
- Include all the keywords pertaining to the section

**Subsection Title (starting timestamp):**
- Briefly describe the key problems or points.
- Mention any significant policies, systems, or events discussed.
- Include all the keywords pertaining to the section

**Another Subsection Title (starting timestamp):**
- Highlight key topics like cultivation, trade, or economic activities.
- Discuss the impact on local communities or any related effects.
- Include all the keywords pertaining to the section

**Subsection Title (starting timestamp):**
- Describe any significant rebellion or uprising.
- Mention the reasons for such events and the consequences they had.
- Include all the keywords pertaining to the section

and so on keep adding more subsections

**Conclusion:**
- Provide a concise summary of the entire chapter, encapsulating the main ideas and impact on the local population.

Each section should have a timestamp indicating when the topic was introduced in the video, followed by a few bullet points that capture the essential details.

# Expected Summary Format Query

Summary Format:

**Introduction (00:00):**
Provide an overview of the main topic of the chapter or video.

**Section Title (Starting Timestamp):**
- Briefly summarize key events or concepts covered in that section, focusing on important details.
- Mention relevant actions, consequences, or systems introduced.
- Mention the section completely in detail

**Conclusion:**
Summarize the overall impact or main takeaway from the entire chapter or video, emphasizing its significance on the broader topic.

The output should be concise, logically structured, and maintain the timestamp references.

# Example Summary for reference on the format(There can be variable number of subsections with variable number of pints under them):

**Introduction (00:00):**
The chapter introduces the British East India Company's involvement in India, focusing on its expansion and control over rural areas, and the economic impacts that followed.

**British East India Company's Control (03:36):**
- The British East India Company gained administrative control over Bengal, Bihar, and Odisha by obtaining Diwani rights from the Mughal ruler in 1765.
- This allowed the Company to directly collect revenue, significantly affecting the economy and lives of the local peasants.

**Revenue Collection and Challenges (05:17):**
- The Company struggled with revenue collection, which led to the introduction of Mahalwari and Ryotwari systems to improve efficiency.
- These systems were meant to streamline taxation and mitigate the grievances of the local farmers.

**Indigo Cultivation and Trade (06:59):**
- Indigo became a major cash crop grown in India and exported to Europe, adding to the Company’s income.
- The cultivation of indigo led to widespread discontent among farmers due to exploitative practices.

**The Blue Rebellion (07:41):**
- The Blue Rebellion, an uprising by the farmers against forced indigo cultivation, is discussed.
- The revolt highlighted the negative impact of British policies on local agriculture and peasant welfare.

**Conclusion:**
The chapter details the British East India Company's strategies for economic control in India, the challenges in revenue collection, and the resulting impact on Indian society, particularly the rural population.

'''
response = rag_chain.invoke({'query': query})['result']

# Output the summary
print("Summary:\n", response)

Summary:
 **Introduction (03:36):**
The chapter delves into the British East India Company's acquisition of administrative control over Bengal, Bihar, and Odisha through the Diwani rights granted by the Mughal ruler in 1765, signifying a significant shift in revenue collection and governance in India.

**British East India Company's Control (03:36):**
- The British East India Company obtained Diwani rights over Bengal, Bihar, and Odisha from the Mughal ruler in 1765, enabling direct revenue collection.
- This transition marked a pivotal moment in Indian history as the Company assumed administrative responsibilities, impacting the local economy and peasant livelihoods significantly.

**Revenue Collection and Challenges (05:17):**
- The Company faced challenges in revenue collection, prompting the introduction of the Mahalwari and Ryotwari systems to streamline taxation processes and address grievances.
- These systems aimed to establish more efficient tax collection mechanisms and impro

# Testing and Validation

In [35]:
!pip install sentence-transformers nltk spacy

In [36]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize
from transformers import pipeline
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
# Load language model
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [37]:
# Sample texts
original_transcript = combined_transcript
generated_summary = response

# Tokenization of sentences
original_sentences = sent_tokenize(original_transcript)
generated_sentences = sent_tokenize(generated_summary)

# TF-IDF Vectorization and Cosine Similarity Calculation
tfidf = TfidfVectorizer().fit_transform([original_transcript, generated_summary])
cosine_sim = cosine_similarity(tfidf, tfidf)[0][1]
print(f"Cosine Similarity between original transcript and summary: {cosine_sim:.2f}")

# Sentence Embeddings Cosine Similarity
embedder = pipeline('feature-extraction', model='bert-base-uncased', truncation=True)

# Function to split text into chunks
MAX_TOKENS = 512
def chunk_text(text, max_tokens):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        length = len(sentence.split())
        if current_length + length <= max_tokens:
            current_chunk.append(sentence)
            current_length += length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = length
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# Split original transcript and summary into chunks
transcript_chunks = chunk_text(original_transcript, MAX_TOKENS)
summary_chunks = chunk_text(generated_summary, MAX_TOKENS)

# Compute embeddings for each chunk and average them
transcript_embeddings = []
summary_embeddings = []

for chunk in transcript_chunks:
    try:
        embedding = np.mean(embedder(chunk, truncation=True, max_length=MAX_TOKENS)[0], axis=0)
        transcript_embeddings.append(embedding)
    except Exception as e:
        print(f"Error processing transcript chunk: {e}")

for chunk in summary_chunks:
    try:
        embedding = np.mean(embedder(chunk, truncation=True, max_length=MAX_TOKENS)[0], axis=0)
        summary_embeddings.append(embedding)
    except Exception as e:
        print(f"Error processing summary chunk: {e}")

# Average all chunk embeddings
if transcript_embeddings:
    transcript_embedding = np.mean(transcript_embeddings, axis=0)
else:
    transcript_embedding = np.zeros((768,))  # Default to zero vector if no embeddings

if summary_embeddings:
    summary_embedding = np.mean(summary_embeddings, axis=0)
else:
    summary_embedding = np.zeros((768,))  # Default to zero vector if no embeddings

# Calculate semantic similarity
semantic_similarity = cosine_similarity([transcript_embedding], [summary_embedding])[0][0]
print(f"Semantic Similarity between transcript and summary: {semantic_similarity:.2f}")

Cosine Similarity between original transcript and summary: 0.46


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Semantic Similarity between transcript and summary: 0.82


# Translation

In [38]:
!pip install translate

In [39]:
from translate import Translator

def translate_section(section, target_lang="hi"):
    translator = Translator(to_lang=target_lang)
    return translator.translate(section)

def split_text(text, max_length=500):
    """Split text into smaller chunks to stay within the character limit."""
    paragraphs = text.split('\n\n')  # Split by paragraphs
    chunks = []
    current_chunk = ""

    for paragraph in paragraphs:
        if len(current_chunk) + len(paragraph) <= max_length:
            current_chunk += paragraph + "\n\n"
        else:
            chunks.append(current_chunk.strip())
            current_chunk = paragraph + "\n\n"

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Split the text into smaller chunks
chunks = split_text(response, max_length=500)

# Translate each chunk
translated_chunks = [translate_section(chunk) for chunk in chunks]

# Combine translated chunks to form the final translation
final_translation = "\n\n".join(translated_chunks)

# Print the final translation
print(final_translation)


**परिचय (03:36 ):**
यह अध्याय 1765 में मुगल शासक द्वारा दिए गए दीवानी अधिकारों के माध्यम से बंगाल, बिहार और ओडिशा पर ब्रिटिश ईस्ट इंडिया कंपनी के प्रशासनिक नियंत्रण के अधिग्रहण के बारे में बताता है, जो भारत में राजस्व संग्रह और शासन में महत्वपूर्ण बदलाव को दर्शाता है।

**ब्रिटिश ईस्ट इंडिया कंपनी का नियंत्रण (03:36 ):**
- ब्रिटिश ईस्ट इंडिया कंपनी ने 1765 में मुगल शासक से बंगाल, बिहार और ओडिशा पर दीवानी अधिकार प्राप्त किए, जिससे प्रत्यक्ष राजस्व संग्रह सक्षम हुआ।
- इस बदलाव ने भारतीय इतिहास में एक महत्वपूर्ण क्षण को चिह्नित किया क्योंकि कंपनी ने स्थानीय अर्थव्यवस्था और किसान आजीविका को महत्वपूर्ण रूप से प्रभावित करते हुए प्रशासनिक जिम्मेदारियां संभालीं।

**राजस्व संग्रह और चुनौतियां (05:17 ):**
- कंपनी को राजस्व संग्रहण में चुनौतियों का सामना करना पड़ा, जिससे कराधान प्रक्रियाओं को कारगर बनाने और शिकायतों को दूर करने के लिए महालवारी और रयोतवारी प्रणालियों की शुरुआत हुई।
- इन प्रणालियों का उद्देश्य अधिक कुशल कर संग्रह तंत्र स्थापित करना और कंपनी और स्थानीय किसानों के बीच संबंधों में सुधा